# Reto 3: El problema final

Este reto consiste en predecir si una imagen contiene o no signos de retinopatía diabética (DR).
Para ello se proporciona un conjunto de datos que contiene características extraídas de imágenes.
Las características extraídas son:
+ [0] Evaluación de la calidad de la imagen , donde 0 = mala calidad 1 = calidad suficiente.
+ [1] El resultado binario de la evaluación previa, donde 1 indica una anormalidad retiniana severa y 0 su falta.
+ [2-7] Los resultados de la detección de microaneurismas (MA). Cada valor de característica representa el número de MAs encontrados en los niveles de confianza alfa = 0.5, ... , 1, respectivamente.
+ [8-15] contienen la misma información que [2-7] para los exudados. <br>
Sin embargo, ya que los exudados están representados por un conjunto de puntos en lugar del número de
píxeles de las lesiones, estas características se normalizan dividiendo los número de lesiones por  el diámetro de la ROI para compensar las diferentes tamaños de las imágenes.
+ [16] La distancia euclidiana del centro dela mácula y el centro del disco óptico para proporcionar información importante con respecto a la condición del paciente. Esta característica también se normaliza con el diámetro de la ROI
+ [17] El diámetro del disco óptico.
+ [18] El resultado binario de la clasificación basada en AM/FM.
+ [19] Etiqueta de clase. 1 = contiene signos de DR

El conjunto de datos, *retinopatia_reto3.csv*, consiste en 592 muestras de imágenes; cada uno de ellos representado por 19 características.

Para evaluar las propuestas se utilizará un conjunto de datos que se mantendrá oculto hasta después de la entrega

### Requisitos
+ **Se debe utilizar algún tipo de reducción de la dimensión o aprendizaje de variedades**
+ A continuación se debe entrenar un clasificador o combinación de clasificadores con las características transformadas según el requisito anterior.
+ Se debe entregar un cuaderno Jupyter con el nombre de los participantes.<br>
  *Por ejemplo*:   **Cuesta_Hinton.ipynb**
+ El cuaderno entregado debe seguir la estructura y reglas de este cuaderno

### Competición
+ Todos los cuadernos entregados se subirán al repo de GitHub y se ejecutarán en Binder, donde ya estará en conjunto de test que estaba oculto.
+ El resultado que se obtenga será la puntuación del reto.
+ **Importante** Es muy fácil asegurarte de que tu código funciona:
    1. Agrupa todo tu código en una única celda
    2. Copialo en el cuaderno del reto que hay en Binder
    3. Ejecuta el cuaderno 
    
### Plazo: lunes 16 de nov. a las 6 am.
Es decir, incluye toda la noche del domingo 15 de nov.
 
> "The final problem", A. C. Doyle (Strand Magazine, diciembre 1983), 
es el relato corto en el que Sherlock Holmes se enfrenta al Prof. Moriarty en las cataratas Reichenbach.

---
    [ES] Código de Alfredo Cuesta Infante para 'Reconocimiento de Patrones'
       @ Master Universitario en Visión Artificial, 2020, URJC (España)
    [EN] Code by Alfredo Cuesta-Infante for 'Pattern Recognition'
       @ Master of Computer Vision, 2020, URJC (Spain)

    alfredo.cuesta@urjc.es

In [1]:
# NO TOCAR ESTA CELDA
# Conjunto distribuido para el reto

Challange_filename = '../../Datasets/retinopatia_reto3.csv'

In [2]:
# NO TOCAR ESTA CELDA
# El conjunto de test cambiará una vez se cierre la entrega
# Ahora mismo es el mismo conjunto que el de entrenamiento

Test_filename = '../../Datasets/retinopatia_test.csv' #<-- este nombre cambiará después del plazo de entrega

In [3]:
# NO TOCAR ESTA CELDA

#-[1]. Load data from CSV and put all in a single dataframe 'FullSet'

import numpy  as np
import pandas as pd
from matplotlib import pyplot as plt
import sys
sys.path.append('../../MyUtils/')
import MyUtils as my
seed = 1234 #<- random generator seed (comment to get randomness)

#-[2]. Load data from CSV and put all in a single dataframe 'FullSet'

FullSet = pd.read_csv(Challange_filename, header=0)
FullX = FullSet.iloc[:,:-1]
FullY = FullSet.iloc[:,-1]

<table style="width:100%;"> 
 <tr style='background:lime'>
  <td style="text-align:left">
      <h2>Tu código debe empezar a partir de aquí y puede tener tantas celdas como quieras</h2>
      <p> Si quieres, puedes borrar (o convertir en RawNBConvert) las celdas de ejemplo
      <h3>Importante:</h3>
      <p>Tu código debe producir las siguientes variables: </p>
      <p> $\quad \bullet$ <b>clf:</b> el clasificador final con el que se realizará el test<br>
       $\quad \bullet$ <b>X_test:</b> el conjunto de test listo para ser usado por el método <b>predict</b><br>
       $\quad \bullet$ <b>Y_test:</b> es el vector de etiquetas del conjunto de X_test listo para ser usado por el método <b>confusion_matrix</b>
      </p>
  </td>
 </tr>
</table>

In [4]:
nombres = ["Antonio Santo", "David Viar"]

In [103]:
#-- preprocesado --

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
valid_size = 0.2
X_train, Y_train, X_valid, Y_valid =    my.single_stratified_split( FullX,FullY, test_size=valid_size, random_state=seed)


scaler = MinMaxScaler()
scaler = StandardScaler()
X = scaler.fit_transform(X_train)
Y = Y_train.values.ravel() 


X_validation = scaler.fit_transform(X_valid)
Y_validation = Y_valid.values.ravel() 

print(X.shape)
print(X_validation.shape)

'''RESULTADO: clf es el objeto con el clasificador'''

(760, 19)
(191, 19)


'RESULTADO: clf es el objeto con el clasificador'

In [104]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
#REDUCCIÓN DE DIMENSIONALIDAD DEL TRAIN
n_components = 1 #<- since there are only 2 classes, this is the only option
lda = LinearDiscriminantAnalysis(n_components=n_components)
lda.fit(X,Y)
X_proy = lda.transform(X)
print(X_proy.shape)
#REDUCCIÓN DE DIMENSIONALIDAD DE LA VALIDACION
lda.fit(X_validation,Y_validation)
X_proy_validation = lda.transform(X_validation)
X_proy_validation.shape

(760, 1)


(191, 1)

In [90]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
ada=AdaBoostClassifier()
search_grid={'n_estimators':[1,10, 100, 200, 300, 400, 500, 600],'learning_rate':[.001,0.01,.1, .2, .3, .4]}
search=GridSearchCV(estimator=ada,param_grid=search_grid,scoring='accuracy',n_jobs=1)
search.fit(X_proy, Y)
search.best_params_

{'learning_rate': 0.001, 'n_estimators': 400}

In [110]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
n_estimators = 400
learning_rate=.001
Ndegree = 3 #<- if = 1 there is no transformatio at all
choice_clf = 'LinearSVC' # 'LinearSVC', 'NBC' , 'DT'
#-LinearSVC is slower other linear clfs. but it allows SAMME.R algorithm
lin_clf  = SVC(kernel='linear',C=0.1, gamma= 'scale', probability=True)
tree_clf = DecisionTreeClassifier(max_depth=1)
nbc_clf  = GaussianNB()
nolin_clf = LogisticRegression()

if choice_clf == 'LinearSVC':
    clf = AdaBoostClassifier(lin_clf,
                      n_estimators=n_estimators, algorithm="SAMME.R", learning_rate=learning_rate)
elif choice_clf == 'DT':
    clf = AdaBoostClassifier(tree_clf,
                      n_estimators=n_estimators, algorithm="SAMME.R", learning_rate=learning_rate)
elif choice_clf == 'NBC':
    clf = AdaBoostClassifier(nbc_clf,
                      n_estimators=n_estimators, algorithm="SAMME.R", learning_rate=learning_rate)
elif choice_clf == 'NOLIN':# Ndegree = 2 #<- if = 1 there is no transformatio at all
    poly_features = PolynomialFeatures(degree=Ndegree, include_bias=False)   
    clf = AdaBoostClassifier(nolin_clf,
                      n_estimators=n_estimators, algorithm="SAMME.R", learning_rate=learning_rate)
    
else:
    print('Choose one base classifier')
 


In [111]:
#SI usaramos el no lineal
#X_poly = poly_features.fit_transform(X_proy)
#print(X_poly.shape)
#clf.fit(X_poly, Y)
#from sklearn.metrics import confusion_matrix
#X_poly_test = poly_features.fit_transform(X_proy_validation)
#print(X_poly_test.shape)

In [112]:
clf.fit(X_proy,Y)

AdaBoostClassifier(base_estimator=SVC(C=0.1, kernel='linear', probability=True),
                   learning_rate=0.001, n_estimators=400)

In [113]:
#EVALUACIÓN CON UN TEST PROPIO DE 191 IMAGENES NUNCA VISTAS

Y_hat = clf.predict(X_proy_validation)
conf_mat = confusion_matrix(Y_validation , Y_hat)
N_success  = np.trace(conf_mat)
N_fails = Y_validation.shape[0]-N_success
#-------------------------------
print (nombres,"\n")
print("Confusion matrix:\n")
print(conf_mat,"\n")
print("Outcome:\n")
strlog = "  :) HIT  = %d, (%0.2f%%)"%(N_success, 100*N_success/(N_success+N_fails))
print(strlog)
strlog = "  :( FAIL = %d, (%0.2f%%)"%(N_fails, 100*N_fails/(N_success+N_fails))
print(strlog)

['Antonio Santo', 'David Viar'] 

Confusion matrix:

[[80 10]
 [33 68]] 

Outcome:

  :) HIT  = 148, (77.49%)
  :( FAIL = 43, (22.51%)


In [114]:
# Preprocesado del test del profesor --

FullSet = pd.read_csv(Test_filename, header=0)
TestX_ = FullSet.iloc[:,:-1]
TestY_ = FullSet.iloc[:,-1]
X2 = scaler.transform(TestX_) 
Y_test = TestY_.values.ravel() 
#REDUCCION DE DIMENSIONALIDAD DE TEST
lda.fit(X2,Y_test)
X_test = lda.transform(X2)
X_test.shape



(951, 1)

<table style="width:100%;"> 
 <tr style='background:pink'>
  <td style="text-align:left">
      <h2>A partir de aquí ya no se pueden modificar las celdas</h2>
          <h3>Comprueba que:</h3>
          <p> $\quad \bullet$ tu clasificador está almacenado en la variable <b>clf</b><br>
              $\quad \bullet$ tienes el conjunto de test correctamente almacenado en la variable <b>X_test</b><br>
              $\quad \bullet$ tienes las etiquetas del conjunto de test correctamente almacenadas en la variable <b>Y_test</b><br>
          </p>
      
  </td>
 </tr>
</table>

## Test

In [115]:
# NO TOCAR ESTA CELDA

from sklearn.metrics import confusion_matrix

Y_hat = clf.predict(X_test)
conf_mat = confusion_matrix(Y_test , Y_hat)
N_success  = np.trace(conf_mat)
N_fails = Y_test.shape[0]-N_success
#-------------------------------
print (nombres,"\n")
print("Confusion matrix:\n")
print(conf_mat,"\n")
print("Outcome:\n")
strlog = "  :) HIT  = %d, (%0.2f%%)"%(N_success, 100*N_success/(N_success+N_fails))
print(strlog)
strlog = "  :( FAIL = %d, (%0.2f%%)"%(N_fails, 100*N_fails/(N_success+N_fails))
print(strlog)

['Antonio Santo', 'David Viar'] 

Confusion matrix:

[[401  45]
 [235 270]] 

Outcome:

  :) HIT  = 671, (70.56%)
  :( FAIL = 280, (29.44%)
